Predicting baby name sex using PyTorch for Logistic Regression

### References

- [Logistic Regression with PyTorch](https://proai.org/pytorch-logistic-regression) by [Denny Loevlie](https://twitter.com/DennisLoevlie)
- [PyTorch OO design compared with SkLearn](https://jeancochrane.com/blog/pytorch-functional-api) by [Jean Cochrane](https://JeanCochrane.com)
- [Building Your First Network in PyTorch](https://t.co/m9I4e0tfrC) by [Ta-Ying Cheng](https://www.linkedin.com/in/tim-ta-ying-cheng-411857139/)
- [data.world US baby names since 1880](https://data.world/ssa/baby-names-for-us-states-territories)



In [1]:
import numpy as np
import pandas as pd
# neither year nor len are statistically significant predictors of sex
from pathlib import Path

In [2]:
pd.options.display.max_rows=7

In [3]:
CWD = Path('.').absolute().resolve()
DATA_DIR_NAME = '.nlpia2-data'
DATA_FILE = 'baby-names-region.csv.gz'
CWD

WindowsPath('C:/Users/maria/Dropbox/Tangible AI/gitlab/nlpia2/src/nlpia2/ch05')

In [4]:
parent = CWD
data_dir = parent / DATA_DIR_NAME 
filepath = data_dir / DATA_FILE
for i in range(10):
    print(filepath)
    if filepath.is_file():
        break
    parent = parent.parent
    data_dir = parent / DATA_DIR_NAME 
    filepath = data_dir / DATA_FILE
filepath

C:\Users\maria\Dropbox\Tangible AI\gitlab\nlpia2\src\nlpia2\ch05\.nlpia2-data\baby-names-region.csv.gz
C:\Users\maria\Dropbox\Tangible AI\gitlab\nlpia2\src\nlpia2\.nlpia2-data\baby-names-region.csv.gz
C:\Users\maria\Dropbox\Tangible AI\gitlab\nlpia2\src\.nlpia2-data\baby-names-region.csv.gz
C:\Users\maria\Dropbox\Tangible AI\gitlab\nlpia2\.nlpia2-data\baby-names-region.csv.gz


WindowsPath('C:/Users/maria/Dropbox/Tangible AI/gitlab/nlpia2/.nlpia2-data/baby-names-region.csv.gz')

In [5]:
def find_dir(dirname=DATA_DIR_NAME, parent=Path('.').absolute().resolve(), max_parents=20):
    for i in range(max_parents):
        data_dir = parent / dirname
        if data_dir.is_dir():
            return data_dir
        parent = parent.parent

DATA_DIR = find_dir()
DATA_DIR

WindowsPath('C:/Users/maria/Dropbox/Tangible AI/gitlab/nlpia2/.nlpia2-data')

In [6]:
def find_file(filename, parent=Path('.').absolute().resolve(), data_dir_name=DATA_DIR_NAME, max_parents=20):
    for i in range(max_parents):
        data_dir = parent / data_dir_name 
        filepath = data_dir / filename
        if filepath.is_file():
            return filepath
        parent = parent.parent

filepath = find_file(DATA_FILE)
filepath

WindowsPath('C:/Users/maria/Dropbox/Tangible AI/gitlab/nlpia2/.nlpia2-data/baby-names-region.csv.gz')

In [7]:
df = pd.read_csv(filepath)

In [8]:
np.random.seed(451)
df = df.sample(10_000)
df

,region,sex,year,name,count,freq
6139665,WV,F,1987,Brittani,10,0.000003
2565339,MD,F,1954,Ida,18,0.000005
22297,AK,M,1988,Maxwell,5,0.000001
...,...,...,...,...,...,...
4475894,OK,F,1950,Leah,9,0.000003
5744351,VA,F,2007,Carley,11,0.000003
5583882,TX,M,2019,Kartier,10,0.000003


In [9]:
names = df['name'].unique()
list(names[:10])

['Brittani',
 'Ida',
 'Maxwell',
 'Charlene',
 'Todd',
 'Aubrey',
 'Arianna',
 'Otis',
 'Trenton',
 'Faustino']

In [10]:
len(names) / len(df)

0.4025

In [11]:
# df = pd.get_dummies(df, columns=['region'])
# df.head()

In [12]:
df = df.rename(dict(name='name_', sex='sex_'), axis=1)
df.head()

,region,sex_,year,name_,count,freq
6139665,WV,F,1987,Brittani,10,0.000003
2565339,MD,F,1954,Ida,18,0.000005
22297,AK,M,1988,Maxwell,5,0.000001
5114650,TN,F,1972,Charlene,24,0.000008
2126395,KS,M,1954,Todd,11,0.000003


In [13]:
df = df.groupby(['name_', 'sex_']).sum()
df

,,year,count,freq
name_,sex_,,,
Aaden,M,2008,51,0.000015
Aahana,F,2018,26,0.000009
Aahil,M,2019,5,0.000002
...,...,...,...,...
Zvi,M,2015,5,0.000002
Zya,F,2019,8,0.000003
Zylah,F,2008,5,0.000001


In [108]:
df_with_duplicates = pd.read_csv(filepath)
np.random.seed(451)
df_with_duplicates = df_with_duplicates.sample(10_000)
df_with_duplicates.set_index(['name', 'sex'])
groups = df_with_duplicates.groupby(['name', 'sex'])

In [113]:
groups = df_with_duplicates.groupby(['name', 'sex'])
for group in groups: 
    if len(group) > 1:
        print (group)

(('Aaden', 'M'),         region sex  year   name  count      freq
1258876     FL   M  2008  Aaden     51  0.000015)
(('Aahana', 'F'),        region sex  year    name  count      freq
612877     CA   F  2018  Aahana     26  0.000009)
(('Aahil', 'M'),         region sex  year   name  count      freq
5584635     TX   M  2019  Aahil      5  0.000002)
(('Aaleyah', 'F'),        region sex  year     name  count      freq
581605     CA   F  2010  Aaleyah     17  0.000005)
(('Aalia', 'F'),        region sex  year   name  count      freq
598948     CA   F  2014  Aalia      7  0.000002
618326     CA   F  2019  Aalia      6  0.000002)
(('Aaliyah', 'F'),         region sex  year     name  count      freq
6034503     WI   F  1998  Aaliyah     18  0.000005
2337545     LA   F  1998  Aaliyah     55  0.000017
3589731     NE   F  2016  Aaliyah     22  0.000007)
(('Aalyah', 'F'),        region sex  year    name  count      freq
546322     CA   F  2001  Aalyah      6  0.000002)
(('Aanya', 'F'),         reg

[8 rows x 6 columns])
(('Alana', 'F'),         region sex  year   name  count      freq
5608995     UT   F  1980  Alana      8  0.000003
4884841     RI   F  1983  Alana      5  0.000002
4767488     PA   F  2011  Alana     73  0.000023
529369      CA   F  1997  Alana    124  0.000039)
(('Alanna', 'F'),         region sex  year    name  count      freq
5352671     TX   F  1998  Alanna     28  0.000009
4615666     OR   F  2007  Alanna      5  0.000001
1337027     GA   F  1988  Alanna      6  0.000002
612341      CA   F  2018  Alanna     67  0.000023
940114      CT   F  2016  Alanna      7  0.000002
1742471     IL   F  1977  Alanna      7  0.000002
3926347     NV   F  2020  Alanna      6  0.000002)
(('Alayah', 'F'),         region sex  year    name  count      freq
6043567     WI   F  2008  Alayah      6  0.000002)
(('Alayna', 'F'),         region sex  year    name  count      freq
6146436     WV   F  2007  Alayna      8  0.000002)
(('Albert', 'M'),         region sex  year    name  count 

(('Amalia', 'F'),        region sex  year    name  count      freq
572699     CA   F  2008  Amalia     32  0.000009)
(('Amanda', 'F'),         region sex  year    name  count      freq
3852428     NM   F  1997  Amanda     79  0.000025
1945399     IN   F  1965  Amanda     51  0.000015
5887415     WA   F  1975  Amanda    179  0.000065
5098154     TN   F  1945  Amanda     10  0.000004)
(('Amani', 'F'),        region sex  year   name  count      freq
938830     CT   F  2014  Amani     11  0.000003)
(('Amarah', 'F'),        region sex  year    name  count      freq
553543     CA   F  2003  Amarah      8  0.000002)
(('Amari', 'F'),         region sex  year   name  count      freq
3220988     MS   F  2013  Amari     22  0.000007)
(('Amariah', 'F'),        region sex  year     name  count      freq
549640     CA   F  2002  Amariah      8  0.000002
565407     CA   F  2006  Amariah     10  0.000003)
(('Amarion', 'M'),         region sex  year     name  count      freq
2416564     LA   M  2006  A

(('Annalie', 'F'),         region sex  year     name  count      freq
5419981     TX   F  2017  Annalie      8  0.000003)
(('Annaliese', 'F'),        region sex  year       name  count      freq
332968     AZ   F  2016  Annaliese      9  0.000003)
(('Annalise', 'F'),         region sex  year      name  count      freq
3085029     MO   F  2006  Annalise     14  0.000004)
(('Annalynn', 'F'),         region sex  year      name  count      freq
1989218     IN   F  2013  Annalynn      5  0.000002)
(('Annamae', 'F'),         region sex  year     name  count      freq
4687591     PA   F  1941  Annamae     12  0.000005)
(('Anne', 'F'),         region sex  year  name  count      freq
5147145     TN   F  2010  Anne     15  0.000005
2677266     ME   F  1958  Anne     37  0.000009
426375      CA   F  1956  Anne    366  0.000093
...        ...  ..   ...   ...    ...       ...
92646       AL   F  2009  Anne     19  0.000006
4255468     OH   F  1912  Anne    111  0.000125
1974614     IN   F  2001  An

(('Asher', 'M'),         region sex  year   name  count      freq
3947511     NV   M  2017  Asher     44  0.000015
706574      CA   M  1992  Asher     17  0.000005)
(('Ashlea', 'F'),         region sex  year    name  count      freq
4061394     NY   F  1995  Ashlea      5  0.000002)
(('Ashlee', 'F'),         region sex  year    name  count      freq
2336074     LA   F  1996  Ashlee     22  0.000007
1747943     IL   F  1981  Ashlee     15  0.000005)
(('Ashleigh', 'F'),         region sex  year      name  count      freq
3089763     MO   F  2010  Ashleigh      7  0.000002
2582291     MD   F  1984  Ashleigh     20  0.000006
4612391     OR   F  2003  Ashleigh      9  0.000003)
(('Ashley', 'F'),         region sex  year    name  count      freq
6198288     WY   F  1994  Ashley     57  0.000017
1374466     GA   F  2014  Ashley     92  0.000029
6029055     WI   F  1991  Ashley    827  0.000236
611890      CA   F  2018  Ashley    358  0.000121
1000585     DC   F  1990  Ashley    158  0.000044


...

(('Sylvia', 'F'),         region sex  year    name  count      freq
3177879     MS   F  1938  Sylvia     84  0.000041
6191804     WY   F  1952  Sylvia      7  0.000002
4683191     PA   F  1935  Sylvia    304  0.000156
5303263     TX   F  1976  Sylvia    205  0.000074)
(('Sylvie', 'F'),        region sex  year    name  count      freq
914186     CT   F  1967  Sylvie      6  0.000002)
(('Symone', 'F'),         region sex  year    name  count      freq
1372215     GA   F  2012  Symone      7  0.000002)
(('Symphony', 'F'),         region sex  year      name  count      freq
4968179     SC   F  2011  Symphony      6  0.000002)
(('Tabatha', 'F'),         region sex  year     name  count      freq
2197749     KY   F  1985  Tabatha     28  0.000009
5712996     VA   F  1973  Tabatha     23  0.000008
3209232     MS   F  1993  Tabatha      5  0.000001)
(('Tabetha', 'F'),         region sex  year     name  count      freq
1126462     FL   F  1990  Tabetha      5  0.000001)
(('Tabitha', 'F'),      

(('Thelma', 'F'),         region sex  year    name  count      freq
4704340     PA   F  1960  Thelma     27  0.000007
6132497     WV   F  1967  Thelma      9  0.000003
3675434     NJ   F  1935  Thelma     29  0.000015
1471908     HI   F  1916  Thelma     21  0.000012)
(('Theo', 'M'),         region sex  year  name  count      freq
5984136     WA   M  2016  Theo     43  0.000014)
(('Theodora', 'F'),         region sex  year      name  count      freq
2915984     MN   F  1939  Theodora      6  0.000003
1686717     IL   F  1917  Theodora     10  0.000005
1086546     FL   F  1952  Theodora      5  0.000001)
(('Theodore', 'M'),         region sex  year      name  count      freq
5933552     WA   M  1913  Theodore     12  0.000012
5062264     SD   M  1935  Theodore     10  0.000005
104381      AL   M  1913  Theodore     26  0.000025
...        ...  ..   ...       ...    ...       ...
5811103     VA   M  2002  Theodore     33  0.000010
5953903     WA   M  1976  Theodore     25  0.000009
19982

(('Troy', 'M'),         region sex  year  name  count      freq
3477782     NC   M  1998  Troy     45  0.000014
3877228     NM   M  1963  Troy     33  0.000009
3665789     NH   M  2019  Troy      6  0.000002
3488627     NC   M  2008  Troy     50  0.000015
1905524     IL   M  2013  Troy     34  0.000011
3004421     MN   M  1999  Troy     21  0.000006
5950659     WA   M  1969  Troy    202  0.000062)
(('Trudy', 'F'),         region sex  year   name  count      freq
1545600     IA   F  1956  Trudy     16  0.000004
5103301     TN   F  1954  Trudy     10  0.000003)
(('Truman', 'M'),        region sex  year    name  count      freq
716971     CA   M  1996  Truman     13  0.000004
768609     CA   M  2015  Truman     21  0.000007)
(('Trystan', 'M'),        region sex  year     name  count      freq
756788     CA   M  2011  Trystan     27  0.000009)
(('Tucker', 'M'),         region sex  year    name  count      freq
2711320     ME   M  1996  Tucker      9  0.000003
2275700     KY   M  2018  Tuck

4988951     SC   M  1942  Wardell      6  0.000002)
(('Waris', 'M'),        region sex  year   name  count      freq
775653     CA   M  2017  Waris      7  0.000002)
(('Warner', 'M'),         region sex  year    name  count      freq
1387464     GA   M  1912  Warner      5  0.000006)
(('Warren', 'M'),         region sex  year    name  count      freq
5933897     WA   M  1915  Warren     20  0.000012
3244485     MS   M  1952  Warren     26  0.000007
3016934     MN   M  2015  Warren     15  0.000005
...        ...  ..   ...     ...    ...       ...
944350      CT   M  1919  Warren     23  0.000012
5197391     TN   M  1988  Warren      9  0.000003
5522726     TX   M  1993  Warren     50  0.000015

[9 rows x 6 columns])
(('Waunita', 'F'),         region sex  year     name  count      freq
6111464     WV   F  1917  Waunita      5  0.000003)
(('Wayland', 'M'),         region sex  year     name  count      freq
5457110     TX   M  1938  Wayland     16  0.000008)
(('Waylon', 'M'),         regi

(('Zakai', 'M'),         region sex  year   name  count      freq
1452893     GA   M  2007  Zakai      5  0.000001)
(('Zakary', 'M'),         region sex  year    name  count      freq
1672177     ID   M  1995  Zakary      6  0.000002
775951      CA   M  2017  Zakary      6  0.000002)
(('Zamora', 'F'),         region sex  year    name  count      freq
1380905     GA   F  2017  Zamora      7  0.000002)
(('Zander', 'M'),         region sex  year    name  count      freq
2716700     ME   M  2018  Zander      8  0.000003)
(('Zandra', 'F'),        region sex  year    name  count      freq
492850     CA   F  1986  Zandra      6  0.000002)
(('Zane', 'M'),         region sex  year  name  count      freq
5013017     SC   M  1993  Zane      8  0.000002
2029141     IN   M  1982  Zane     11  0.000003
1668528     ID   M  1981  Zane      5  0.000002)
(('Zaniyah', 'F'),         region sex  year     name  count      freq
4348762     OH   F  2007  Zaniyah      5  0.000001
3410130     NC   F  2012  Zani

In [14]:
df['name'] = df.index.get_level_values('name_')
df['sex'] = df.index.get_level_values('sex_')
df.head()

,,year,count,freq,name,sex
name_,sex_,,,,,
Aaden,M,2008,51,0.000015,Aaden,M
Aahana,F,2018,26,0.000009,Aahana,F
Aahil,M,2019,5,0.000002,Aahil,M
Aaleyah,F,2010,17,0.000005,Aaleyah,F
Aalia,F,4033,13,0.000004,Aalia,F


In [15]:
df.query('name == "Chris"')

year  count      freq   name sex
name_ sex_                                  
Chris F     1983      5  0.000002  Chris   F
      M     7850    239  0.000069  Chris   M

In [114]:
counts[('Timothy',)]

sex
F       5
M    3538
Name: count, dtype: int64

In [16]:
df['istrain'] = np.random.rand(len(df)) < .9
df.head()

,,year,count,freq,name,sex,istrain
name_,sex_,,,,,,
Aaden,M,2008,51,0.000015,Aaden,M,True
Aahana,F,2018,26,0.000009,Aahana,F,True
Aahil,M,2019,5,0.000002,Aahil,M,True
Aaleyah,F,2010,17,0.000005,Aaleyah,F,True
Aalia,F,4033,13,0.000004,Aalia,F,True


In [17]:
# A list of dicts or a dict of dicts is fastest way to create dataframe from groups of rows
# https://stackoverflow.com/users/8727339/mohit-motwani
# https://stackoverflow.com/a/57001947/623735

df_most_common = {}
for name, group in df.groupby('name'):
    row_dict = group.iloc[group['count'].argmax()].to_dict()
    df_most_common[(name, row_dict['sex'])] = row_dict
df_most_common = pd.DataFrame(df_most_common).T
df_most_common

,,year,count,freq,name,sex,istrain
Aaden,M,2008,51,0.000015,Aaden,M,True
Aahana,F,2018,26,0.000009,Aahana,F,True
Aahil,M,2019,5,0.000002,Aahil,M,True
...,...,...,...,...,...,...,...
Zvi,M,2015,5,0.000002,Zvi,M,True
Zya,F,2019,8,0.000003,Zya,F,True
Zylah,F,2008,5,0.000001,Zylah,F,True


In [71]:
df = df_with_duplicates;
for name, group in df_with_duplicates.groupby('name'):
    if len(group) > 1: 
        df.drop(group['count'].idxmin(), inplace=True)

In [66]:
df = df_with_duplicates
group = df.groupby('name').get_group('Chris')
group['count'].idxmax()

('Chris', 'M')

In [72]:

df.query('name == "Chris"')

,,year,count,freq,name,sex,istrain
name_,sex_,,,,,,
Chris,M,7850,239,0.000069,Chris,M,True


In [19]:
df_most_common['istest'] = ~df_most_common['istrain'].astype(bool)
print(df_most_common['name sex count istrain istest'.split(' ')])

            name sex count istrain  istest
Aaden  M   Aaden   M    51    True   False
Aahana F  Aahana   F    26    True   False
Aahil  M   Aahil   M     5    True   False
...          ...  ..   ...     ...     ...
Zvi    M     Zvi   M     5    True   False
Zya    F     Zya   F     8    True   False
Zylah  F   Zylah   F     5    True   False

[4025 rows x 5 columns]


In [77]:
df_most_common[['istest', 'istrain']].sum() / len(df_most_common)

istest     0.095652
istrain    0.904348
dtype: float64

In [21]:
istest = df_most_common['istest']
istest

Aaden   M    False
Aahana  F    False
Aahil   M    False
             ...  
Zvi     M    False
Zya     F    False
Zylah   F    False
Name: istest, Length: 4025, dtype: bool

In [22]:
istest.sum()

385

In [23]:
istest_idx = df_most_common[istest].index
istest_idx[:4]

MultiIndex([('Abelardo', 'M'),
            ( 'Adaline', 'F'),
            ( 'Adalynn', 'F'),
            ( 'Addelyn', 'F')],
           )

In [24]:
df['istrain'].sum() / len(df)

0.9043478260869565

In [25]:
df['istest'] = df_most_common['istest']
df['istest'] = df['istest'].fillna(False)
df['istrain'] = ~df['istest']
df['istrain'].sum() / len(df)

0.9043478260869565

In [26]:
df[~df['istrain'] & ~df['istest']]

,,year,count,freq,name,sex,istrain,istest


In [80]:
istrain = df['istrain']
istrain.sum() / len(istrain)

0.9040993788819875

In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(analyzer='char', ngram_range=(1, 3), lowercase=False)
vectorizer

TfidfVectorizer(analyzer='char', lowercase=False, ngram_range=(1, 3))

In [81]:
unique_names = df['name'][istrain].unique()
vectorizer.fit(unique_names)
vecs = vectorizer.transform(df['name'])
vecs

<4025x3657 sparse matrix of type '<class 'numpy.float64'>'
	with 57656 stored elements in Compressed Sparse Row format>

In [82]:
vecs = pd.DataFrame(vecs.toarray())
vecs.columns = vectorizer.get_feature_names_out()
vecs.index = df.index
print(vecs.iloc[:,:5])

                    A        Aa       Aad       Aah  Aal
name_  sex_                                             
Aaden  M     0.193989  0.393903  0.505031  0.000000  0.0
Aahana F     0.183496  0.372597  0.000000  0.454943  0.0
Aahil  M     0.186079  0.377841  0.000000  0.461346  0.0
...               ...       ...       ...       ...  ...
Zvi    M     0.000000  0.000000  0.000000  0.000000  0.0
Zya    F     0.000000  0.000000  0.000000  0.000000  0.0
Zylah  F     0.000000  0.000000  0.000000  0.000000  0.0

[4025 rows x 5 columns]


In [31]:
import torch
torch

<module 'torch' from 'C:\\Users\\maria\\.conda\\envs\\nlpia2\\lib\\site-packages\\torch\\__init__.py'>

In [32]:
class LogisticRegressionNN(torch.nn.Module):

    def __init__(self, num_features, num_outputs=1):
         super().__init__()
         self.linear = torch.nn.Linear(num_features, num_outputs)

    def forward(self, X):
        return torch.sigmoid(self.linear(X))

In [33]:
def make_tensor(X):
    """ Convert numpy ndarray to torch.Tensor """
    X = getattr(X, 'values', X)
    return X if isinstance(X, torch.Tensor) else torch.Tensor(X)

def make_array(x):
    """ Convert torch.Tensor to numpy 1-D array """
    if hasattr(x, 'detach'):
        return torch.squeeze(x).detach().numpy()
    return x

In [34]:
num_features = vecs.shape[1]  # number of unique n-grams in our "vocabulary"
num_outputs = 1    # number of nesses (sexes) to predict, we're predicting only femaleness

In [35]:
from tqdm import tqdm
import time
import json
import copy

# Fraction of the tensors y_pred and y that are the same 
# (y_pred == y).sum() / len(y)
def measure_binary_accuracy(y_pred, y):
    """ Round y_pred and y then count the preds that are equal to the truth to compute fraction correct """
    y_pred = make_array(y_pred).round()
    y = make_array(y).round()
    num_correct = (y_pred == y).sum()
    return num_correct / len(y)

In [36]:
def measure_performance(model, X_train, X_test, y_train, y_test, criterion):
    with torch.no_grad():
        # Calculating the loss and accuracy for the train dataset
        accuracy_train = measure_binary_accuracy(model(X_train), y_train)
        outputs_test = torch.squeeze(model(X_test))
        accuracy_test = measure_binary_accuracy(outputs_test, y_test)
        loss_test = criterion(outputs_test, y_test)
        return dict(i=i, 
                    # loss_train=loss.item(),
                    accuracy_train=accuracy_train,
                    loss_test=loss_test.item(),
                    accuracy_test=accuracy_test)

In [37]:
model = LogisticRegressionNN(num_features=vecs.shape[1], num_outputs=1)
model

LogisticRegressionNN(
  (linear): Linear(in_features=3657, out_features=1, bias=True)
)

In [38]:
learning_rate = 0.05
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
optimizer

SGD (
Parameter Group 0
    dampening: 0
    lr: 0.05
    momentum: 0
    nesterov: False
    weight_decay: 0
)

In [44]:
df[['count']].columns

Index(['count'], dtype='object')

In [73]:
# BCE: Binary Cross Entropy
criterion = torch.nn.BCELoss(weight=torch.Tensor(df[['count']].values))
criterion

BCELoss()

In [84]:
X = vecs.values
y = (df[['sex']] == 'F').values
X_train = torch.Tensor(X[istrain])
X_test = torch.Tensor(X[~istrain])
y_train = torch.Tensor(y[istrain])
y_test = torch.Tensor(y[~istrain])

In [95]:
num_epochs = 200
loss_func_train = torch.nn.BCELoss(weight=torch.Tensor(df[['count']][istrain].values))
hyperparam_values ={'lr': 0.02,'momentum': 0.001}
model = LogisticRegressionNN(num_features=vecs.shape[1], num_outputs=1)
optimizer = torch.optim.SGD(model.parameters(), **hyperparam_values)
pbar_epochs = tqdm(range(num_epochs), desc='Epoch:', total=num_epochs)

for epoch in pbar_epochs:
    optimizer.zero_grad() # Setting our stored gradients equal to zero
    outputs = model(X_train)
    loss_train = loss_func_train(outputs, y_train) 
    loss_train.backward() # Computes the gradient of the given tensor w.r.t. the weights/bias
    optimizer.step() # Updates weights and biases with the optimizer (SGD)

Epoch:: 100%|██████████| 200/200 [00:01<00:00, 138.24it/s]


In [118]:
model = LogisticRegressionNN(num_features=vecs.shape[1], num_outputs=1)
optimizer = torch.optim.SGD(model.parameters(), **hyperparam_values)
loss_func_test = torch.nn.BCELoss(weight=torch.Tensor(df[['count']][~istrain].values)) 

for epoch in range(num_epochs):
    optimizer.zero_grad() 
    outputs = model(X_train)
    loss_train = loss_func_train(outputs, y_train) 
    loss_train.backward() 
    optimizer.step() 
    outputs_test = model(X_test)
    loss_test = loss_func_test(outputs_test, y_test)
    accuracy_test = measure_binary_accuracy(outputs_test, y_test)
    if epoch % 20 == 19:
        print((f'Epoch {epoch}: loss_train/test: {loss_train.item():.4f}/{loss_test.item():.4f}, accuracy_test: {accuracy_test:.4f}'))

Epoch 19: loss_train/test: 80.1816/75.3989, accuracy_test: 0.4275
Epoch 39: loss_train/test: 75.0748/74.4430, accuracy_test: 0.5933
Epoch 59: loss_train/test: 71.0529/73.7784, accuracy_test: 0.6503
Epoch 79: loss_train/test: 67.7637/73.2873, accuracy_test: 0.6839
Epoch 99: loss_train/test: 64.9957/72.9028, accuracy_test: 0.6891
Epoch 119: loss_train/test: 62.6145/72.5862, accuracy_test: 0.6995
Epoch 139: loss_train/test: 60.5302/72.3139, accuracy_test: 0.7073
Epoch 159: loss_train/test: 58.6803/72.0716, accuracy_test: 0.7073
Epoch 179: loss_train/test: 57.0198/71.8502, accuracy_test: 0.7202
Epoch 199: loss_train/test: 55.5152/71.6437, accuracy_test: 0.7280


In [105]:
num_epochs = 100
#hyperparam_values = {'lr': 0.289}
hyperparam_values ={'lr': 0.02,'momentum': 0.001}
model = LogisticRegressionNN(num_features=vecs.shape[1], num_outputs=1)
optimizer = torch.optim.SGD(model.parameters(), **hyperparam_values)
criterion_train = torch.nn.BCELoss(weight=torch.Tensor(df[['count']][istrain].values))  # BCE: Binary Cross Entropy
criterion_test = torch.nn.BCELoss(weight=torch.Tensor(df[['count']][~istrain].values))  # BCE: Binary Cross Entropy
X = vecs.values
y = (df[['sex']] == 'F').values
X_train = torch.Tensor(X[istrain])
X_test = torch.Tensor(X[~istrain])
y_train = torch.Tensor(y[istrain])
y_test = torch.Tensor(y[~istrain])

pbar_epochs = tqdm(range(num_epochs), desc='Epoch:', total=num_epochs)
results = [None] * num_epochs
for epoch in range(num_epochs):
    optimizer.zero_grad() # Setting our stored gradients equal to zero
    outputs = model(X_train)
    loss_train = criterion_train(outputs, y_train) 
    loss_train.backward() # Computes the gradient of the given tensor w.r.t. the weights/bias
    epoch_loss_train = loss_train.item()
    optimizer.step() # Updates weights and biases with the optimizer (SGD)
    #print(f'Train loss: {np.round(loss_train.detach().numpy(), 4):0.4f}')
    outputs_test = model(X_test)
    loss_test = criterion_test(outputs_test, y_test).item()
    accuracy_test = measure_binary_accuracy(outputs_test, y_test)
    #results[epoch] = dict(loss_train=loss_train, loss_test=loss_test, accuracy_test=accuracy_test)
    if epoch % 20 == 19:
        print((f'Epoch {epoch}: loss_train/test: {loss_train:.4f}/{loss_test:.4f}, accuracy_test: {accuracy_test}'))
    #pbar_epochs.set_description(f'loss_train/test: {loss_train:.4f}/{loss_test:.4f}')
print((f'loss_train/test: {loss_train:.4f}/{loss_test:.4f}, accuracy_test: {accuracy_test}'))

Epoch::   0%|          | 0/100 [28:01<?, ?it/s]


Epoch 19: loss_train/test: 80.2628/75.2845, accuracy_test: 0.43005181347150256
Epoch 39: loss_train/test: 75.1420/74.3274, accuracy_test: 0.6139896373056994
Epoch 59: loss_train/test: 71.1092/73.6625, accuracy_test: 0.6580310880829016
Epoch 79: loss_train/test: 67.8115/73.1716, accuracy_test: 0.6839378238341969
Epoch 99: loss_train/test: 65.0367/72.7877, accuracy_test: 0.6839378238341969
Epoch 119: loss_train/test: 62.6499/72.4720, accuracy_test: 0.6968911917098446
Epoch 139: loss_train/test: 60.5611/72.2008, accuracy_test: 0.7020725388601037
Epoch 159: loss_train/test: 58.7074/71.9598, accuracy_test: 0.7124352331606217
Epoch 179: loss_train/test: 57.0437/71.7398, accuracy_test: 0.7202072538860104
Epoch 199: loss_train/test: 55.5364/71.5347, accuracy_test: 0.727979274611399
loss_train/test: 55.5364/71.5347, accuracy_test: 0.727979274611399


In [ ]:
# Create new majority sex column to hold the most common sex for names used for both sexes
df['majority_sex'] = df['sex']

# Overwrite the minority sex with the majority.
for name_, sex_ in df_most_common.index:
    opposite_sex = 'F' if sex_ == 'M' else 'M'
    try:
        # if you put majority_sex after .iloc this fails to overwrite data and generates warning
        df['majority_sex'].loc[(name_, opposite_sex)] = sex_
        print(f'overwrote ({name_}, {opposite_sex}) with {sex_}')
    except KeyError:
        pass


In [ ]:
dupe_names = df[df['name'].duplicated()]['name'].values
df[df['name'].isin(dupe_names)]

In [ ]:
weight_test = torch.Tensor(df[['count']].loc[df_most_common.index].values)
criterion_test = torch.nn.BCELoss(weight=weight_test)
criterion_test

In [ ]:
def rand_range(min_value=0.001, max_value=1):
    scale = max_value - min_value
    return scale * np.random.rand() + min_value

In [ ]:
def rand_range_log(min_value=0.001, max_value=1):
    min_log = np.log(min_value)
    max_log = np.log(max_value)
    return np.exp(rand_range(np.log(min_value), np.log(max_value)))

Create random hyperparameter table for optimizer learning_rate and momentum

In [ ]:
# lr: learning_rate
hyperparam_ranges = dict(lr=[0.02, 1.0], momentum=[0.00001, 1.0])
hyperparam_table = []
num_attempts = 30
for i in range(num_attempts):
    hyperparam_values = dict()
    for k, v in hyperparam_ranges.items():
        hyperparam_values[k] = rand_range_log(*hyperparam_ranges[k])
    hyperparam_table.append(hyperparam_values)
pd.DataFrame(hyperparam_table)

In [ ]:
model = LogisticRegressionNN(num_features=vecs.shape[1], num_outputs=1)
model

In [ ]:
optimizer = torch.optim.SGD(model.parameters(), **hyperparam_table[0])
optimizer

In [ ]:
weight_train = torch.Tensor(df['count'][istrain].values.reshape(-1, 1) / df['count'][istrain].mean())
weight_train.sum()

In [ ]:
df['count'][istrain].values.reshape(sum(istrain), 1)

In [ ]:
weight_train.shape

In [ ]:
weight_test = torch.Tensor(df['count'][~istrain].values.reshape(-1, 1) / df['count'][~istrain].mean())
weight_test.sum()

In [ ]:
# pbar = tqdm(hyperparam_table, desc='Training attempt', total=len(hyperparam_table))
num_epochs=200

t0 = time.time()
for i, hyperparam_values in enumerate(hyperparam_table):
    t1 = time.time()
    model = LogisticRegressionNN(num_features=vecs.shape[1], num_outputs=1)
    optimizer = torch.optim.SGD(model.parameters(), lr=hyperparam_values['lr'])
    # BCE: Binary Cross Entropy weighted by the number of babies with that first name and sex
    criterion_train = torch.nn.BCELoss(weight=weight_train)
    criterion_test = torch.nn.BCELoss(weight=weight_test)
    X = vecs.values
    X_train = torch.Tensor(X[istrain])
    X_test = torch.Tensor(X[~istrain])
    y_train = torch.Tensor((df[['sex']] == 'F').astype(int).values[istrain])
    y_test = torch.Tensor((df[['majority_sex']] == 'F').astype(int).values[~istrain])
    
    pbar_epochs = tqdm(range(num_epochs), desc='Epoch:', total=num_epochs)
    results = [None] * num_epochs
    for epoch in pbar_epochs:
        optimizer.zero_grad() # Setting our stored gradients equal to zero
        outputs = model(X_train)
        loss_train = criterion_train(outputs, y_train) 
        loss_train.backward() # Computes the gradient of the given tensor w.r.t. the weights/bias
        loss_train = loss_train.item()
        optimizer.step() # Updates weights and biases with the optimizer (SGD)
        # print(f'Train loss: {np.round(loss_train.detach().numpy(), 4):0.4f}')
        outputs_test = model(X_test)
        loss_test = criterion_test(outputs_test, y_test).item()
        accuracy_test = measure_binary_accuracy(outputs_test, y_test)
        results[epoch] = dict(loss_train=loss_train, loss_test=loss_test, accuracy_test=accuracy_test)
        # pbar_epochs.set_description(f'loss_train/test: {loss_train:.4f}/{loss_test:.4f}')
    t2 = time.time()
    results[-1]['time_per_attempt'] = t2 - t1
    results[-1]['total_time'] = t2 - t0
    hyperparam_table[i].update(results[-1])
    print(f'attempt {i}/{len(hyperparam_table)}')
    for k, v in hyperparam_table[i].items():
        print(f'{k}: {v:04f}')

In [ ]:
istest = df['istest']
df['name'][istest].unique()


In [ ]:
hyperparam_table[-1]




In [ ]:
hyperparam_df = pd.DataFrame(hyperparam_table).sort_values('accuracy_test')
hyperparam_df

In [ ]:
def fit(model=model, X=vecs.values, y=(df[['sex']] == 'F').values, optimizer=None,
        num_epochs=30, learning_rate=.1, criterion=criterion, optimizer=optimizer):
    pbar = tqdm(range(num_epochs), desc='Epoch', total=num_epochs)
    X_train = torch.Tensor(X[istrain])
    X_test = torch.Tensor(X[~istrain])
    y_train = torch.Tensor(y[istrain])
    y_test = torch.Tensor(y[~istrain])

    results = []
    for i in pbar:
        optimizer.zero_grad() # Setting our stored gradients equal to zero
        outputs = model(X_train)
        loss_train = criterion(outputs, y_train) 
        loss_train.backward() # Computes the gradient of the given tensor w.r.t. the weights/bias
        optimizer.step() # Updates weights and biases with the optimizer (SGD)
    return results

In [ ]:
test_loss = hyperparam_df['loss_test']
test_accuracy = hyperparam_df['accuracy_test']
hyperparam_df.plot(x='lr', y='momentum', kind='scatter', logy=True, logx=True, grid=True, marker='o', s=100*test_accuracy**2, alpha=test_loss*.7)

In [ ]:
results = fit()

In [ ]:
pd.DataFrame(results)

In [ ]:
# model.score(vecs[~istrain], y[~istrain], sample_weight=df['count'][~istrain])

In [ ]:
# model.classes_


In [ ]:
names = ['Dewey', 'Kemal', 'Copeland', 'Vishvesh']
ourvecs = vectorizer.transform(names)
ourvecs = pd.DataFrame.sparse.from_spmatrix(ourvecs)
ourvecs.columns = vectorizer.get_feature_names_out()
ourvecs.index = list(zip(names, 'M'*len(names)))
ourvecs

In [ ]:
ourtensors = 

In [ ]:
names = ['Maria', 'Syndee', 'Aditi', 'Constance']
vecs = vectorizer.transform(names)
vecs = pd.DataFrame.sparse.from_spmatrix(vecs)
vecs.columns = vectorizer.get_feature_names_out()
vecs.index = list(zip(names, 'M'*len(names)))
pd.DataFrame(model.predict_proba(vecs)[:,0], index=vecs.index)

In [ ]:
class LogisticRegressionNumpyNN(LogisticRegressionNN):

    def __init__(self, *args, **kwargs):
         super().__init__(*args, **kwargs)

    def predict_proba(self, X):
        return self.forward(make_tensor(X))
    
    def predict(self, X):
        return (np.array(self.forward(make_tesnor(X))) > 0.5).astype(int)
    
# ', '.join([v for v in dir(LogisticRegression) if v[0] != '_'])